In [1]:
import numpy as np 
import pandas as pd 
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale
import random
import cv2
import warnings

warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Activation,  MaxPool2D, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation
from tensorflow.keras.optimizers import Adam

In [3]:
# The path of the baseline code has been modified
training_folder = '../input/k-pop-idol-face-identification-competition/train/'
test_folder = '../input/k-pop-idol-face-identification-competition/test/'
train_label = pd.read_csv('../input/k-pop-idol-face-identification-competition/train_label.csv')
train_label["filepath"] = training_folder + train_label["file_name"]
train_label = train_label.drop(["file_name"], axis=1)

In [4]:
train_label.name.unique()

array(['lisa', 'jisoo', 'rose', 'kimminju', 'jennie', 'joyuri'],
      dtype=object)

In [5]:
name_convert_dict = {
    'lisa' : 0,
    'joyuri' : 1,
    'rose' : 2,
    'jisoo' : 3,
    'kimminju' : 4,
    'jennie' : 5,
}

In [6]:
train_label["name"] = train_label.name.map(lambda x: name_convert_dict[x])

In [7]:
train_label = shuffle(train_label, random_state=42) #데이터를 무작위로 섞음
train_size = int(len(train_label)*0.8) # 훈련에 사용할 데이터의 크기를 지정
training_df = train_label[:train_size] # 훈련 데이터셋을 만들어줌
validation_df = train_label[train_size:] # validation 데이터셋을 만들어줌

In [8]:
batch_size = 8 # 배치 사이즈를 설정
image_size = 512 # 이미지의 크기를 설정
input_shape = (image_size, image_size, 3) #이미지의 사이즈 정의 (컬러 이미지이기 때문에 한 화소당 3개의 데이터가 필요)
dropout_rate = 0.3 #드롭아웃 비율 정의
classes_to_predict = sorted(training_df.name.unique()) #예측해야 하는 클래스 수 정의, 여기서는 6개

In [9]:
training_data = tf.data.Dataset.from_tensor_slices((training_df.filepath.values, training_df.name.values))
validation_data = tf.data.Dataset.from_tensor_slices((validation_df.filepath.values, validation_df.name.values))

In [10]:
def load_image_and_label_from_path(image_path, label): #이미지 데이터를 불러와 텐서 (array와 비슷한 형태)로 변환하는 함수
    img = tf.io.read_file(image_path) #이미지 경로의 파일을 읽음
    img = tf.image.decode_jpeg(img, channels=3) #이미지를 array 데이터로 변환하여 저장
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE #메모리 동적 할당을 위한 AUTOTUNE
training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE) #train 데이터를 불러옴
validation_data = validation_data.map(load_image_and_label_from_path,num_parallel_calls=AUTOTUNE) #validation 데이터를 불러옴

In [11]:
#train 및 validation 데이터를 훈련하기 좋게 batch로 자름
training_data_batches = training_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
validation_data_batches = validation_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [12]:
data_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"), #랜덤으로 이미지를 좌우로 뒤집어줌.
        layers.experimental.preprocessing.RandomRotation(0.25), #이미지를 좌우로 25% 이내로 랜덤으로 돌립니다. 
        layers.experimental.preprocessing.RandomZoom((-0.2, 0.2)), #이미지를 0~20%만큼 랜덤으로 축소,확대합니다.
        layers.experimental.preprocessing.RandomContrast(factor=0.2),
        layers.experimental.preprocessing.RandomHeight(factor=0.2),
        layers.experimental.preprocessing.RandomWidth(factor=0.2)
        
    ]
)

In [13]:
efficientnet = EfficientNetB0(weights="imagenet", #이미지넷 가중치 값을 불러와 적용
                              include_top=False, 
                              input_shape=input_shape, 
                              drop_connect_rate=dropout_rate) #efficientnetB0 모델을 로드
efficientnet.trainable=True # efficientnetb0의 학습을 허용. 만약 False로 지정할 시에 정확도는 떨어지지만 학습 속도가 매우 빨라짐

In [14]:
model = Sequential() #새 Sequential 모델을 만듬 
model.add(Input(shape=input_shape)) #인풋을 이미지 사이즈로 설정
model.add(data_augmentation_layers) #이미지 augumentation 레이어 추가
model.add(efficientnet) # efficientnetb0 추가

model.add(Conv2D(64, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=5,strides=3,padding="same"))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))

model.add(layers.GlobalAveragePooling2D())  # 풀링 레이어를 추가

model.add(Dense(640, activation = "relu"))
model.add(layers.Dropout(dropout_rate))     # 드롭아웃 레이어를 추가

model.add(Dense(len(classes_to_predict), activation="softmax")) #마지막 덴스 레이어를 추가. 예측할 클래스의 개수만큼이 아웃풋이 된다. 
model.summary() #모델 확인

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 512, 512, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 16, 16, 1280)     4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                      

In [ ]:
epochs = 30 #에폭 수를 설정합니다.
decay_steps = int(round(len(training_df)/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3) #learning rate를 에폭이 지날수록 점점 줄여나가는 cosine decay 방법을 사용합니다. 
callbacks = [ModelCheckpoint(filepath='mymodel.h5', monitor='val_loss', save_best_only=True), #가장 validation loss가 낮은 에폭의 모델을 .h5 파일로 저장합니다. 
            EarlyStopping(monitor='val_loss', patience = 5, verbose=1)] #정해진 에폭이 되기 전에 5번의 에폭동한 validation loss가 향상되지 않으면 학습을 종료합니다. 

model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(cosine_decay), metrics=["accuracy"]) #loss는 sparse_categorical_crossentropy, optimizer는 Adam을 사용합니다. 각 에폭당 정확도를 통해 모델의 성능을 모니터링합니다, 

In [ ]:
history = model.fit(training_data_batches, #모델을 학습합니다. 
                  epochs = epochs, 
                  validation_data=validation_data_batches,
                  callbacks=callbacks)

In [ ]:
sample_submission = pd.read_csv("../input/k-pop-idol-face-identification-competition/sample_submission.csv")

In [ ]:
sample_submission.head()

In [ ]:
def load_image_test(image_name):
    test_folder = "../input/k-pop-idol-face-identification-competition/test/"
    image_path = "../input/k-pop-idol-face-identification-competition/test/" + image_name
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = np.reshape(img, [-1,512,512,3])
    return img

In [ ]:
def test_predict(file_name):
    local_image = load_image_test(file_name) #함수를 통하여 이미지를 불러옴
    predictions = model.predict(local_image) #각 클래스의 확률을 훈련한 모델으로 예측
    final_prediction = np.argmax(predictions) #가장 확률이 높은 클래스를 최종 예측 결과로 도출
    return final_prediction #예측결과 반환

In [ ]:
def predictions_over_image(filepath):
    predictions = [] #에측값을 저장하는 리스트
    for path in filepath:   
        predictions.append(test_predict(path)) # 각 사진마다 예측값을 저장한다
    return predictions #예측값 변환

In [ ]:
sample_submission["name"] = predictions_over_image(sample_submission["file_name"])
sample_submission["name"] = sample_submission.name.map(lambda x : list(name_convert_dict.keys())[list(name_convert_dict.values()).index(x)])

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
from IPython.display import FileLink
FileLink('submission.csv')